<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [2]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 58 kB/s 
     |████████████████████████████████| 200 kB 66.4 MB/s 
     |████████████████████████████████| 199 kB 73.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=345ad7e475220430f1e014f80019ebb160cca26b5ad7af1230021f54c2ed9b24
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-19 10:00:41--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [3]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [4]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [5]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-19 10:01:04--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 108.139.29.105, 108.139.29.7, 108.139.29.58, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|108.139.29.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236526966 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.57M   148MB/s    in 1.5s    

2022-11-19 10:01:05 (148 MB/s) - ‘name.basics.tsv.gz’ saved [236526966/236526966]

--2022-11-19 10:01:05--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 108.139.29.105, 108.139.29.7, 108.139.29.58, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|108.139.29.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287698034 (274M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.3

##Rozpakowanie danych

In [6]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [7]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [8]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00718...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0057345,tt00491...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [9]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [10]:
#df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw różnicę w różnych typach łączenia tabel df_name_basics_selected i df_title_principals_selected:

In [11]:
from timeit import default_timer as timer

start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53092579, 8)
joined table size:  (53086246, 12) time of joining tables:  102.31950027199991


In [12]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53092579, 8)
joined table size:  (53092579, 12) time of joining tables:  95.3393988769999


In [13]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53100409, 8)
joined table size:  (53094038, 12) time of joining tables:  93.49155305000022


In [14]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53100409, 8)
joined table size:  (53100409, 12) time of joining tables:  91.08722057499972


In [15]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53092579, 8)
joined table size:  (53086246, 12) time of joining tables:  102.11611766800024


In [16]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53092579, 8)
joined table size:  (53092579, 12) time of joining tables:  92.30456612099988


In [17]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53100409, 8)
joined table size:  (53094038, 12) time of joining tables:  92.0158353959996


In [18]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53100409, 8)
joined table size:  (53100409, 12) time of joining tables:  89.841880855


In [19]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53094038, 8)
joined table size:  (53086246, 12) time of joining tables:  80.33504936500003


In [20]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53094038, 8)
joined table size:  (53094038, 12) time of joining tables:  79.50506735700037


In [21]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53100409, 8)
joined table size:  (53092579, 12) time of joining tables:  78.15886678799961


In [22]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53100409, 8)
joined table size:  (53100409, 12) time of joining tables:  79.75223640900003


In [23]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53094038, 8)
joined table size:  (53086246, 12) time of joining tables:  77.75515627999994


In [24]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53094038, 8)
joined table size:  (53094038, 12) time of joining tables:  77.19903446900025


In [25]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53100409, 8)
joined table size:  (53092579, 12) time of joining tables:  83.26864368799943


In [26]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53100409, 8)
joined table size:  (53100409, 12) time of joining tables:  79.86181255200063


---------------------------------------------------------------------------------------------------

In [27]:
df_name_basics = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

In [28]:
start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20160298, 8)
joined table size:  (20156715, 12) time of joining tables:  69.4937123210002


In [29]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20160298, 8)
joined table size:  (20160298, 12) time of joining tables:  69.72700742399957


In [ ]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20783276, 8)


In [ ]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

------------------------------------------------------------------------------------------------

In [ ]:
print(data9.filter(data9.primaryName.isNull()).count())
print(data9.filter(data9.category.isNull()).count())
print(data9.filter(data9.genres.isNull()).count())

In [ ]:
print(data11.filter(data11.primaryName.isNull()).count())
print(data11.filter(data11.category.isNull()).count())
print(data11.filter(data11.genres.isNull()).count())

In [ ]:
print(data12.filter(data12.primaryName.isNull()).count())
print(data12.filter(data12.category.isNull()).count())
print(data12.filter(data12.genres.isNull()).count())

In [ ]:
print(data13.filter(data13.primaryName.isNull()).count())
print(data13.filter(data13.category.isNull()).count())
print(data13.filter(data13.genres.isNull()).count())

In [ ]:
print(data14.filter(data14.primaryName.isNull()).count())
print(data14.filter(data14.category.isNull()).count())
print(data14.filter(data14.genres.isNull()).count())

In [ ]:
print(data15.filter(data15.primaryName.isNull()).count())
print(data15.filter(data15.category.isNull()).count())
print(data15.filter(data15.genres.isNull()).count())

In [ ]:
print(data16.filter(data16.primaryName.isNull()).count())
print(data16.filter(data16.category.isNull()).count())
print(data16.filter(data16.genres.isNull()).count())

Tutaj za każdym razem wychodzą różne wyniki, dlatego ciężko stwierdzić, który sposób łączenia danych jest najszybszy, jednak zauważamy, że jedynie dwa sposoby ("inner" i "left") dają odpowiednie dane, gdyż otrzymujemy wtedy dane o aktorach z pełnymi danymi personalnymi bez zbędnych wierszy nieprzypisanych do żadnego imienia i nazwiska.

In [ ]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample